In [ ]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

First, we read in the .csv files as pandas DataFrames. And we plot the locations of the stations.

In [ ]:
df = pd.read_csv("../data/PRECIP_QUANTITY_SETHY_PRECIP_all_stat_1950101-20240101.csv")
df_meta = pd.read_csv("../data/sethy_station_meta.csv")

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
ax.scatter(df_meta.lon, df_meta.lat)

Next, we process the DataFrames and convert them to xarray DataSets. 

In [ ]:
ds_meta = xr.Dataset.from_dataframe(df_meta)

ds_raw = xr.Dataset.from_dataframe(df)
ds = ds_raw.swap_dims(index="time")
ds["time"] = np.array(ds.time, dtype="datetime64[ns]")
ds = ds.set_coords(["year", "month", "day", "hour"])

locations = list(ds.data_vars)
da_combined = xr.concat([ds[loc] for loc in locations], dim="location")
da_combined = da_combined.assign_coords(location=locations)

ds = xr.Dataset({"pr": da_combined})
ds

Now, we take a look at the period of the Vesder case. We find the locations with the five highest accumulated precipitation sums. We save this dataset for the students.

In [ ]:
ds_period = ds.sel(time=slice("2021-07-12T01", "2021-07-17T00"))
pr_sum = ds_period.pr.sum("time")
pr_sum.location.sortby(pr_sum)

In [ ]:
max_locations = ["Jalhay", "Spa", "Mont-Rigi", "Neu-Hattlich", "Erezee"]
ds_period.sel(location=max_locations).pr.plot(hue="location")

In [ ]:
ds_meta_locs_list = list()

for location in max_locations:
    ds_meta_loc = ds_meta.where(ds_meta.name == location).dropna("index")
    ds_meta_locs_list.append(ds_meta_loc)

ds_meta_locs = xr.concat(ds_meta_locs_list, dim="index")

fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.coastlines(alpha=0.7)
ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
ax.scatter(ds_meta_locs.lon, ds_meta_locs.lat)
plt.show()

In [ ]:
ds_sel = ds.sel(time=slice("2021-07-12T01", "2021-07-17T00")).sel(
    location=max_locations
)
lons = list()
lats = list()

for location in ds_sel.location:
    ds_meta_loc = ds_meta.where(ds_meta.name == location).dropna("index")
    lons.append(ds_meta_loc.lon.values[0])
    lats.append(ds_meta_loc.lat.values[0])

ds_sel = ds_sel.assign_coords(lon=("location", lons), lat=("location", lats))
ds_sel.attrs = {
    "standard_name": "precipitation_flux",
    "long_name": "Precipitation",
    "units": "mm/h",
    "cell_methods": "time: mean",
}
ds_sel.to_netcdf("../data/SETHY_pr_2021-07-12T01_2021-07-17T00.nc", engine="netcdf4")